# Firestore PoC

Demonstrates python functionality for taking data from races and aggregating it in the appropriate collections in Firestore. Can also be used to add sample data to our database.

## Initialize Cloud Firestore SDK on Google Cloud

In [52]:
import os
from dotenv import load_dotenv
from pathlib import Path
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

env_path = Path('..')/'.env'
load_dotenv(dotenv_path=env_path)

if not firebase_admin._apps:
    cred = credentials.Certificate(os.environ.get("GOOGLE_APPLICATION_CREDENTIALS"))
    firebase_admin.initialize_app(cred)
    
db = firestore.client()

## Utility Functions

In [2]:
import re

def getDocId(name: str):
    cleaned = re.sub(r"[/?*=#]", "", name)
    return cleaned.replace(" ", "_").lower()

## Race and Result Class

In [34]:
from typing import List

class Result:
    def __init__(self, netid: str, name: str, college: str, place: int, points: int, lap1: float, lap2: float, lap3: float):
        self.netid = netid
        self.name = name
        self.college = college

        self.college_id = getDocId(college)
        
        self.place = place
        self.points = points
        self.lap1 = lap1
        self.lap2 = lap2
        self.lap3 = lap3
        
        self.total_time = lap1 + lap2 + lap3

    @staticmethod
    def from_dict(source):
        return Result(
            netid = source["netid"],
            name = source["name"],
            college = source["college"],
            place = source["place"],
            points = source["points"],
            lap1 = source["lap1"],
            lap2 = source["lap2"],
            lap3 = source["lap3"]
        )

    def to_dict(self):
        return {
            "netid": self.netid,
            "name": self.name,
            "college": self.college,
            "college_id": self.college_id,
            "place": self.place,
            "points": self.points,
            "lap1": self.lap1,
            "lap2": self.lap2,
            "lap3": self.lap3,
            "total_time": self.total_time
        }

    def __repr__(self):
        return (f"---\n\n"
                f"NetId: {self.netid}\n"
                f"Name: {self.name}\n"
                f"College: {self.college} ({self.college_id})\n"
                f"Place: {self.place}\n"
                f"Points: {self.points}\n"
                f"Total Time: {self.total_time}\n"
                f"Splits: {self.lap1} | {self.lap2} | {self.lap3}\n\n")

class Race:
    def __init__(self, datetime: str, event: str, race_number: int, track: str, results: List[Result]):
        self.datetime = datetime
        self.event = event

        self.event_id = getDocId(event)
        
        self.race_number = race_number

        self.race_id = getDocId(f"{event}-{str(race_number).zfill(3)}")
        
        self.track = track
        self.results = sorted(results, key = lambda result: result.total_time)

    @staticmethod
    def from_dict(source):
        results = [Result.from_dict(result) for result in source["results"]]
        return Race(
            datetime = source["datetime"],
            event = source["event"],
            race_number = source["race_number"],
            track = source["track"],
            results = results
        )

    def to_dict(self):
        results = [result.to_dict() for result in self.results]
        return {
            "datetime": self.datetime,
            "event": self.event,
            "event_id": self.event_id,
            "race_number": self.race_number,
            "race_id": self.race_id,
            "track": self.track,
            "results": results
        }

    def __repr__(self):
        return (f"Datetime: {self.datetime}\n"
                f"Event: {self.event} ({self.event_id})\n"
                f"Race Number: {self.race_number} ({self.race_id})\n"
                f"Track: {self.track}\n"
                f"Results:\n\n"
                f"{''.join([str(result) for result in self.results])}")

## Create Sample Race Data

In [37]:
datetime = "2025-06-15T1:12:27"

event = "Trumbull Circuit #1"
race_number = 6

track = "Baby Park"

results = [
    {
        "netid": "ref42",
        "name": "Ryan Fernandes",
        "college": "Trumbull",
        "place": 1,
        "points": 15,
        "lap1": 70.89,
        "lap2": 60.54,
        "lap3": 65.66
    },
    {
        "netid": "oad9",
        "name": "Oscar de la Cerda",
        "college": "Ezra Stiles",
        "place": 3,
        "points": 12,
        "lap1": 74.89,
        "lap2": 72.11,
        "lap3": 60.10,
    },
    {
        "netid": "ds3249",
        "name": "David Sadka",
        "college": "Timothy Dwight",
        "place": 4,
        "points": 11,
        "lap1": 71.89,
        "lap2": 75.99,
        "lap3": 70.22
    },
    {
        "netid": "dkl26",
        "name": "Daniel Landry",
        "college": "Berkeley",
        "place": 12,
        "points": 1,
        "lap1": 79.89,
        "lap2": 70.23,
        "lap3": 80.12,
    }
]

result_objects = [Result.from_dict(result) for result in results]
race_object = Race(
                datetime = datetime,
                event = event,
                race_number = race_number,
                track = track,
                results = result_objects
                )

In [38]:
race_object

Datetime: 2025-06-15T1:12:27
Event: Trumbull Circuit #1 (trumbull_circuit_1)
Race Number: 6 (trumbull_circuit_1-006)
Track: Baby Park
Results:

---

NetId: ref42
Name: Ryan Fernandes
College: Trumbull (trumbull)
Place: 1
Points: 15
Total Time: 197.09
Splits: 70.89 | 60.54 | 65.66

---

NetId: oad9
Name: Oscar de la Cerda
College: Ezra Stiles (ezra_stiles)
Place: 3
Points: 12
Total Time: 207.1
Splits: 74.89 | 72.11 | 60.1

---

NetId: ds3249
Name: David Sadka
College: Timothy Dwight (timothy_dwight)
Place: 4
Points: 11
Total Time: 218.1
Splits: 71.89 | 75.99 | 70.22

---

NetId: dkl26
Name: Daniel Landry
College: Berkeley (berkeley)
Place: 12
Points: 1
Total Time: 230.24
Splits: 79.89 | 70.23 | 80.12


## Post Data to Races Collection

In [40]:
db.collection("races").document(race_object.race_id).set(race_object.to_dict())

update_time {
  seconds: 1750008294
  nanos: 895767000
}

## Post Data to Users Collection

In [75]:
# Implement college rank later

import requests

YALIES_KEY = os.environ.get("YALIES_KEY")

def findYalie(netid):
    headers = {
    	"Authorization": f"Bearer {YALIES_KEY}"
    }
    body = {
    	"filters": {"netid": netid}
    }
    request = requests.post("https://api.yalies.io/v2/people", headers=headers, json=body)
    return request.json()

def validateYalie(yalie):
    if (len(yalie) > 0):
        yalie_object = yalie[0]
    else:
        raise RuntimeError(f"Yalies response empty: {yalie}")
    
    if ("netid" in yalie_object) and ("first_name" in yalie_object) and ("last_name" in yalie_object) and ("year" in yalie_object) and ("college" in yalie_object):
        return {
            "netid": yalie_object["netid"],
            "first_name": yalie_object["first_name"],
            "last_name": yalie_object["last_name"],
            "year": yalie_object["year"],
            "college": yalie_object["college"]
        }
    else:
        raise RuntimeError(f"Yalies response incomplete: {yalie_object}")

def createNewUser(netid: str):
    yalies_data = validateYalie(findYalie(netid))
    college_id = getDocId(yalies_data["college"])
    user_data = {
        "netid": yalies_data["netid"],
        "first_name": yalies_data["first_name"],
        "last_name": yalies_data["last_name"],
        "year": yalies_data["year"],
        "college": yalies_data["college"],
        "points": 0,
        "race_count": 0,
        "races": []
    }

    return db.collection("users").document(user_data["netid"]).set(user_data)

In [76]:
createNewUser("ref42")

update_time {
  seconds: 1750011825
  nanos: 577677000
}